In [1]:
import pandas as pd

In [2]:
# Load files as df
data_1 = pd.read_csv('Milestone 1 Dataset/hr_temp_20170201-20200131_subset.csv')
data_2 = pd.read_csv('Milestone 1 Dataset/hrl_load_metered - 20170201-20200131.csv')

In [3]:
# Create data frame with only variables of interest
data = {'MW': data_2['mw'], 'Temp': data_1['HourlyDryBulbTemperature'], 'Date': data_1['DATE']}
data_df = pd.DataFrame(data)
data_df.head()

,MW,Temp,Date
0,1419.881,37.0,2017-02-01T00:53:00
1,1379.505,37.0,2017-02-01T01:53:00
2,1366.106,36.0,2017-02-01T02:53:00
3,1364.453,36.0,2017-02-01T03:53:00
4,1391.265,36.0,2017-02-01T04:53:00


In [4]:
# Cast as datetime
data_df['Date'] = pd.to_datetime(data_df['Date'])   

In [5]:
# Create time variables out of the 'Date' variable
data_df['hour'] = [x.hour for x in data_df['Date']]  #hour
data_df['dow'] =  [x.weekday() for x in data_df['Date']] #day of week
data_df['month'] =  [x.month for x in data_df['Date']] #month 
data_df['year'] = [x.year for x in data_df['Date']] #year

In [6]:
# Set 'Date' as index
data_df.set_index('Date', inplace=True)

In [7]:
# Fill in missing values in the 'Temp' variable by linear interpolation
data_df.interpolate(method='linear', inplace=True)
sum(data_df['Temp'].isnull())

0

In [8]:
# Divide data into training (until 12-31-2019) and testing (since January, 2020)
training = data_df[:'12-31-2019']
testing = data_df['01-01-2020':]

In [9]:
training.head()

,MW,Temp,hour,dow,month,year
Date,,,,,,
2017-02-01 00:53:00,1419.881,37.0,0,2,2,2017
2017-02-01 01:53:00,1379.505,37.0,1,2,2,2017
2017-02-01 02:53:00,1366.106,36.0,2,2,2,2017
2017-02-01 03:53:00,1364.453,36.0,3,2,2,2017
2017-02-01 04:53:00,1391.265,36.0,4,2,2,2017


In [10]:
testing.head()

,MW,Temp,hour,dow,month,year
Date,,,,,,
2020-01-01 00:53:00,1363.428,31.0,0,2,1,2020
2020-01-01 01:53:00,1335.975,29.0,1,2,1,2020
2020-01-01 02:53:00,1296.817,30.0,2,2,1,2020
2020-01-01 03:53:00,1288.403,30.0,3,2,1,2020
2020-01-01 04:53:00,1292.263,31.0,4,2,1,2020
